In [2]:
import pandas as pd
import numpy as np
import math
from datetime import datetime
from sklearn.linear_model import LinearRegression

In [17]:
df_small = pd.read_csv('data/datafest2018NewApril6_CANADA_SMALL.csv')

In [3]:
df = pd.read_csv('data/datafest2018NewApril6.csv')

In [3]:
# US Sample data
df = pd.read_csv('data/sample1.csv')
df.shape

(100000, 25)

In [95]:
df['normTitleCategory'].columns.values

AttributeError: 'Series' object has no attribute 'columns'

In [30]:
listings = df_small.groupby(['companyId', 'jobId'])['date'].apply(', '.join).reset_index()
listings.head(10)

companyId     jobId     
company00003  job0001121                            2016-10-06, 2016-10-10
              job0005179                                        2016-11-06
              job0010571                            2016-11-21, 2016-11-28
              job0021975                            2017-01-31, 2017-03-08
              job0023964                            2017-03-04, 2017-02-03
              job0025943                                        2017-04-03
              job0026922    2017-03-13, 2017-03-21, 2017-03-31, 2017-03-14
              job0054913                2017-08-31, 2017-06-25, 2017-09-03
              job0056115                                        2017-08-05
              job0067749                            2017-09-09, 2017-09-07
Name: date, dtype: object

In [6]:
# Generate Days Posted for each listing
def date_difference(row):
    end_day = datetime.strptime(row['EndDate'], '%Y-%m-%d')
    start_day = datetime.strptime(row['PostDate'], '%Y-%m-%d')
    return (end_day-start_day).days + 1

listings = df.groupby(['companyId', 'jobId'])
earliest_date = listings['date'].min().reset_index(name='PostDate')
latest_date = listings['date'].max().reset_index(name='EndDate')
click_count = listings['clicks'].sum().reset_index(name='Clicks')
dates = earliest_date.merge(latest_date).merge(click_count)
dates['DaysPosted'] = dates.apply(date_difference, axis=1)
print("Finished calculating")
dates = dates.merge(df, on=['companyId', 'jobId'])
print("Finished joining")
dates = dates[['city', 'stateProvince', 'Clicks', 'PostDate', 'DaysPosted']].drop_duplicates()
dates = dates.rename(index=str, columns={'Clicks': 'TotalClicks', 'DaysPosted': 'AvgDays'})
city_listings = dates.groupby(['city', 'stateProvince']).agg({'TotalClicks': 'sum', 'AvgDays': 'mean'}).reset_index()
print("Finished aggregating")
city_listings.head(20)

Finished calculating
Finished joining
Finished aggregating


,city,stateProvince,TotalClicks,AvgDays
0,29 Palms,CA,60,1.000000
1,Abbeville,LA,111,1.000000
2,Abbeville,SC,9,1.000000
3,Abbotsford,WI,27,1.000000
4,Abbott,TX,15,1.000000
5,Abbott Park,IL,81,1.000000
6,Aberdeen,MD,537,7.400000
7,Aberdeen,NC,55,1.000000
8,Aberdeen,NJ,18,1.000000
9,Aberdeen,SD,256,2.666667


In [8]:
city_listings.shape

(9102, 4)

In [9]:
city_listings.to_csv('data/listing_time.csv')
print("Finished writing to csv")

Finished writing to csv


In [90]:
# Regression
lm = LinearRegression(n_jobs=-1)
independent = df[['']]

KeyError: ('clicks', 'occurred at index Unnamed: 0')